### 1) Initial Imports and loading the utils function. The dataset is used is <a href='https://www.kaggle.com/adityajn105/flickr8k'>Flickr 30k</a> from kaggle.<br>Custom dataset and dataloader is implemented in <a href="https://www.kaggle.com/mdteach/torch-data-loader-flicker-8k">this</a> notebook.

In [ ]:
#location of the data
data_location =  "../input/flickr30k"
!ls $data_location

#reading the text data
import pandas as pd
caption_file = data_location + '/captions.txt'
df = pd.read_csv(caption_file)
print("There are {} image to captions".format(len(df)))
df.head(7)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#select any index from the whole dataset
#single image has 5 captions
#so, select indx as: 1,6,11,16...
data_idx = 11

#eg path to be plot: ../input/flickr8k/Images/1000268201_693b08cb0e.jpg
image_path = data_location+"/Images/"+df.iloc[data_idx,0]
img=mpimg.imread(image_path)
plt.imshow(img)
plt.show()

#image consits of 5 captions,
#showing all 5 captions of the image of the given idx
for i in range(data_idx,data_idx+5):
    print("Caption:",df.iloc[i,1])


<h2>2) Writing the custom dataset</h2>
<p>Writing the custom torch dataset class so, that we can abastract out the dataloading steps during the training and validation process</p>
<p>Here, dataloader is created which gives the batch of image and its captions with following processing done:</p>

<li>caption word tokenized to unique numbers</li>
<li>vocab instance created to store all the relivent words in the datasets</li>
<li>each batch, caption padded to have same sequence length</li>
<li>image resized to the desired size and converted into captions</li>

<br><p>In this way the dataprocessing is done, and the dataloader is ready to be used with <b>Pytorch</b></p>

In [ ]:
!python -m spacy download en_core_web_sm
import os
from collections import Counter
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

from PIL import Image

#using spacy for the better text tokenization
spacy_eng = spacy.load("en_core_web_sm")

#example
text = "Let's learn deep learning"
tokens = [token.text.lower() for token in spacy_eng.tokenizer(text)]
print(tokens)


***xây dựng vocabulary cho bài toán***

In [ ]:
class Vocabulary:
    def __init__(self,freq_threshold):
        #setting the pre-reserved tokens int to string tokens
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}

        #string to int tokens
        #its reverse dict self.itos
        self.stoi = {v:k for k,v in self.itos.items()}

        self.freq_threshold = freq_threshold # tần suất xuất hiện tối thiểu để thêm 1 từ vào từ điển

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_eng.tokenizer(str(text))]

    def build_vocab(self, sentence_list):
        frequencies = Counter() # hàm counter giúp đếm số lượng từ xuất hiện
        idx = 4 # số tiếp theo của từ điển do đã có sẵn 0123

        for sentence in sentence_list:
            for word in self.tokenize(sentence):
                frequencies[word] += 1

                #add the word to the vocab if it reaches minum frequecy threshold
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self,text):
        """ For each word in the text corresponding index token for that word form the vocab built as list """
        tokenized_text = self.tokenize(text)
        return [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]

In [ ]:
#testing the vicab class
v = Vocabulary(freq_threshold=1)

v.build_vocab(["This is a good place to find a city"])
print(v.stoi)
print(v.numericalize("This is a good place to find a city here!!"))

****Thiết lập dataset để truy cập lấy ảnh và caption đồng thời chuyển caption về dạng numerical tensor thông qua vocab đã xây dựng bên trên****

In [ ]:
class FlickrDataset(Dataset):

    def __init__(self,root_dir,captions_file,transform=None,freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(caption_file)
        self.transform = transform

        #Get image and caption colum from the dataframe
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        #Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions.tolist()) # build vocab từ caption


    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        caption = self.captions[idx] # lấy caption dựa trên index
        img_name = self.imgs[idx] # lấy tên ảnh dựa trên index

        # tạo đường dẫn đến tệp ảnh dựa vào tên ảnh đã có và đổi về dạng RGB
        img_location = os.path.join(self.root_dir,img_name)
        img = Image.open(img_location).convert("RGB")

        #apply the transfromation to the image
        if self.transform is not None:
            img = self.transform(img)

        #numericalize the caption text
        caption_vec = []
        caption_vec += [self.vocab.stoi["<SOS>"]]
        caption_vec += self.vocab.numericalize(caption)
        caption_vec += [self.vocab.stoi["<EOS>"]]

        return img, torch.tensor(caption_vec) # img là ảnh đã biến đổi về RGB
                                              # caption_vec là tensor numerical của caption thông qua vocab

In [ ]:
#defing the transform to be applied
transforms = T.Compose([
    T.Resize((224,224)),
    T.ToTensor() # chuyển về tensor và chuẩn hóa [0,1]
])

In [ ]:
def show_image(inp, title=None):
    """Imshow for Tensor."""
    # chuyển tensor về numpy và đinh nghĩa lại thứ tự các chiều để plot
    inp = inp.numpy().transpose((1, 2, 0)) # đổi c,h,w thành h,w,c (1,2,0)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
#testing the dataset class
dataset =  FlickrDataset(
    root_dir = data_location+"/Images",
    captions_file = data_location+"/captions.txt",
    transform=transforms
)
if '.' in dataset.vocab.stoi:
    # Lấy chỉ số của token '.' trong từ điển stoi
    idx = dataset.vocab.stoi['.']

    # Xóa token '.' khỏi stoi và itos
    del dataset.vocab.stoi['.']
    del dataset.vocab.itos[idx]

for i in range(10):
    print(dataset.captions[i])

In [ ]:
img, caps = dataset[5]
show_image(img,"Image")
print("Token:",caps)
print("Sentence:")
print([dataset.vocab.itos[token] for token in caps.tolist()])

***Padding và chuyển về tensor 4 chiều***

In [ ]:
# hàm giúp padding các caption và chuyển về tensor 4 chiếu
class CapsCollate:
    """
    Collate to apply the padding to the captions with dataloader
    """
    def __init__(self,pad_idx,batch_first=False):
        self.pad_idx = pad_idx # pad_idx là giá trị số dùng để padding
        self.batch_first = batch_first
        # nếu batch_first = True thì kích cỡ ma trận numerical của 1 caption có kích cỡ batch, lenght.
        # False thì ngược lại

    def __call__(self,batch): # batch là tập tuple(image,caption) của một batch
        imgs = [item[0].unsqueeze(0) for item in batch] # thêm 1 chiều cho ảnh (C, H, W) sẽ trở thành (1, C, H, W)
        imgs = torch.cat(imgs,dim=0) # ghép tất cả ảnh thành 1 tensor duy nhất (B, C, H, W) với B là số ảnh trong batch

        targets = [item[1] for item in batch] # 1 list các tensor caption 1 chiều
        targets = pad_sequence(targets, batch_first=self.batch_first, padding_value=self.pad_idx) # padding
        return imgs,targets # trả về tensor 4d của ảnh và 2d của caption (batch, max_length)

In [ ]:

# load data
def get_data_loader(dataset,batch_size=4,shuffle=False,num_workers=1):
    """
    Returns torch dataloader for the flicker8k dataset

    Parameters
    -----------
    dataset: FlickrDataset (which was created above)
        custom torchdataset named FlickrDataset
    batch_size: int
        number of data to load in a particular batch
    shuffle: boolean,optional;
        should shuffle the datasests (default is False)
    num_workers: int,optional
        numbers of workers to run (default is 1)
    """

    pad_idx = dataset.vocab.stoi["<PAD>"] # giá trị số của token padding
    collate_fn = CapsCollate(pad_idx=pad_idx,batch_first=True) # padding

    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=collate_fn
    )

    return data_loader


In [ ]:
#location of the training data
data_location =  "../input/flickr30k"

#imports
import numpy as np
import torch
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T


### 2) **<b>Implementing the Helper function to plot the Tensor image**

In [ ]:
#show the tensor image bằng cách chuyển tensor về dạng trước khi chuẩn hóa rồi plot
import matplotlib.pyplot as plt
def show_image(img, title=None):
    """Imshow for Tensor."""

    #unnormalize
    img[0] = img[0] * 0.229
    img[1] = img[1] * 0.224
    img[2] = img[2] * 0.225
    img[0] += 0.485
    img[1] += 0.456
    img[2] += 0.406

    img = img.numpy().transpose((1, 2, 0))


    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
#Initiate the Dataset and Dataloader

#setting the constants
data_location =  "../input/flickr30k"
BATCH_SIZE = 128
NUM_WORKER = 4

#defining the transform to be applied
transforms = T.Compose([
    T.Resize(226),  # Resize the image to
    T.RandomCrop(224),  # Randomly crop a 224x224 patch
    T.ToTensor(), # Chuyển đổi ảnh sang tensor
    T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) # Chuẩn hóa giá trị pixel với giá trị mean và std
])


#thiết lập dataset TT bên trên nhưng khác phép biến đổi
dataset =  FlickrDataset(
    root_dir = data_location+"/Images",
    captions_file = data_location+"/captions.txt",
    transform=transforms
)


# tạo data_loader bằng hàm đã thiết lập bên trên
data_loader = get_data_loader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=True,
    # batch_first = True
)

#vocab_size
vocab_size = len(dataset.vocab)
print(vocab_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

### 3) Defining the Model Architecture

Model is seq2seq model. In the **encoder** pretrained ResNet model is used to extract the features. Decoder, is the implementation of the Bahdanau Attention Decoder. In the decoder model **LSTM cell**.

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

In [ ]:

class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        # Load Inception v3 with pretrained weights
        inception = models.inception_v3(pretrained = True)

        for param in inception.parameters():
            param.requires_grad = False

        if hasattr(inception, 'AuxLogits'):
            inception.AuxLogits = None

        # Keep all layers except the fully connected and average pooling layers
        self.inception = nn.Sequential(*list(inception.children())[:-3])  # Retain up to the convolutional layers

    def forward(self, images):
        # Forward pass through the Inception model
        features = self.inception(images)  # (batch_size, 2048, 8, 8)
        features = features.permute(0, 2, 3, 1)  # Change to (batch_size, 8, 8, 2048) for compatibility
        features = features.view(features.size(0), -1, features.size(-1))
        return features

In [ ]:
'''Bahdanau Attention: đầu tiên ta cần tính điểm score bằng công thức sau:
Ei,t = A . tanh(U . fi + W . ht) với fi là output resnet, ht là hidden state thứ t của decode
U,W là các ma trận trọng số cần học, A là vecto trọng số. Các tham số này cố định tại mọi t và i

fi là vecto đặc trưng thứ i trong ma trận đặc trưng của RESNET. Ta biết output RESNET có dạng 49,2048 (49 row, 2048 column)
==> có 49 vecto đặc trưng f với mỗi vecto có kích cỡ 2048

ht là hidden state tại timestep t của lớp decode LSTM.

Ei,t là 1 score đánh giá độ liên quan giữa state hiện tại của decode với vecto đặc trưng ảnh tương ứng.
Ta cần cho nó đi qua hàm softmax để chuẩn hóa về khoảng [0,1], ta được tham số alpha tương ứng với từng vecto đặc trưng
(có 49 alpha do có 49 vecto đặc trưng) ==> Alpha thay đổi theo từng state ht khác nhau

Tại bước cuối, ta cần nhân alpha với vecto đặc trưng tương ứng rồi cộng tổng tất cả lại, ta được vecto đặc trưng ảnh
ft với t tại timestep t và vecto này đã tập trung vào các phần model cần dự đoán trong state t+1 của decode

'''

class Attention(nn.Module):
    def __init__(self, encoder_dim,decoder_dim,attention_dim):
        super(Attention, self).__init__()

        self.attention_dim = attention_dim


        # tính attetion score
        self.W = nn.Linear(decoder_dim,attention_dim)  # chuyển hidden state ra không gian attention
        self.U = nn.Linear(encoder_dim,attention_dim)   # chuyển vecto đặc trưng ra không gian attention
        self.A = nn.Linear(attention_dim,1) # ánh xạ không gian attention thành score






    def forward(self, features, hidden_state):
        u_hs = self.U(features)     #(batch_size,num_layers,attention_dim)
        w_ah = self.W(hidden_state) #(batch_size,attention_dim)

        combined_states = torch.tanh(u_hs + w_ah.unsqueeze(1)) #(batch_size,num_layers,attemtion_dim)

        attention_scores = self.A(combined_states)         #(batch_size,num_layers,1)
        attention_scores = attention_scores.squeeze(2)     #(batch_size,num_layers)


        alpha = F.softmax(attention_scores,dim=1)          #(batch_size,num_layers)

        attention_weights = features * alpha.unsqueeze(2)  #(batch_size,num_layers,features_dim)
        attention_weights = attention_weights.sum(dim=1)   #(batch_size,num_layers)

        return alpha,attention_weights


In [ ]:
#Attention Decoder
class DecoderRNN(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()

        #save the model param
        self.vocab_size = vocab_size
        self.attention_dim = attention_dim
        self.decoder_dim = decoder_dim

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.attention = Attention(encoder_dim,decoder_dim,attention_dim)


        self.init_h = nn.Linear(encoder_dim, decoder_dim) # khởi tạo hidden state
        self.init_c = nn.Linear(encoder_dim, decoder_dim) # khởi tạo cell state
        self.lstm_cell = nn.LSTMCell(embed_size+encoder_dim,decoder_dim,bias=True)  # khởi tạo LSTM với 2 input lààà
                                                                        # embedding vecto và vecto đặc trưng qua attetion



        self.f_beta = nn.Linear(decoder_dim, encoder_dim) # tạo gating mechanism, giúp quản lý các gate trong LSTM


        self.fcn = nn.Linear(decoder_dim,vocab_size) # tầng fcn để ánh xạ từ hidden state sang không gian vocab
        self.drop = nn.Dropout(drop_prob)



    def forward(self, features, captions):

        #vectorize the caption
        embeds = self.embedding(captions)

        # Initialize LSTM state
        h, c = self.init_hidden_state(features)  # (batch_size, decoder_dim)

        #get the seq length to iterate
        seq_length = len(captions[0])-1     # độ dài của câu trừ token end
        batch_size = captions.size(0) # số ảnh trong batch
        num_features = features.size(1) # số vecto đặc trưng (49)

        # khởi tạo tensor chứa kết quả dự đoán dưới dạng softmax tại mỗi timestep
        preds = torch.zeros(batch_size, seq_length, self.vocab_size).to(device)
        alphas = torch.zeros(batch_size, seq_length,num_features).to(device)
        #  (batch_size, seq_length, num_features) là kích cỡ alphas dùng để lưu tất cả alpha trong attetion

        for s in range(seq_length):
            alpha,context = self.attention(features, h)
            lstm_input = torch.cat((embeds[:, s], context), dim=1) # nối embedding với context vecto lại thành input LSTM
            h, c = self.lstm_cell(lstm_input, (h, c))

            output = self.fcn(self.drop(h)) # lớp fcn giúp chuyển hidden state mới nhất thành vecto output để đoán từ tiếp
            # output có kích cỡ (batch_size, vocab_size) với mỗi dòng là kết quả softmax các từ trong bộ từ điển

            # lưu lại các giá trị vừa có vào preds và alphas
            preds[:,s] = output
            alphas[:,s] = alpha


        return preds, alphas # tensor các softmax tại mỗi timestep và tensor các alpha tại mỗi timestep


    # using beam search
    def generate_caption(self, features, max_len=20, vocab=None, beam_size=3):
        # Inference part using beam search with log-probabilities
        batch_size = features.size(0)
        h, c = self.init_hidden_state(features)  # (batch_size, decoder_dim)

        alphas = []

        # Starting input (the <SOS> token)
        word = torch.tensor(vocab.stoi['<SOS>']).view(1, -1).to(device)
        embeds = self.embedding(word)

        # Initialize beams with the first token and its probability
        beams = [(embeds, h, c, [vocab.stoi['<SOS>']], 0)]  # (input embedding, h, c, list of previous word indices, score)

        for step in range(max_len):  # At each timestep
            all_candidates = []

            # Expand each beam by generating the next word
            for embeds, h, c, seq, score in beams:  # Iterate over each beam
                # If this beam has finished (contains <EOS>), just add it to all_candidates without generating new word
                if seq and seq[-1] == vocab.stoi['<EOS>']:
                    all_candidates.append((embeds, h, c, seq, score))
                    continue

                # Calculate attention and context for each beam
                alpha, context = self.attention(features, h)
                alphas.append(alpha.cpu().detach().numpy())

                lstm_input = torch.cat((embeds[:, 0], context), dim=1)
                h, c = self.lstm_cell(lstm_input, (h, c))
                output = self.fcn(self.drop(h))  # (batch_size, vocab_size)
                output = output.view(batch_size, -1)

                # Calculate log-probabilities of the next word
                log_probs = torch.log_softmax(output, dim=1)  # Convert scores to log-probabilities

                # Select the top beam_size words with the highest log-probabilities
                topk_log_probs, topk_indices = torch.topk(log_probs, beam_size, dim=1)

                for i in range(beam_size):
                    # For each candidate, append the word and score
                    word_idx = topk_indices[:, i]
                    new_log_prob = topk_log_probs[:, i].item()  # Take the log probability of the word
                    new_score = score + new_log_prob  # Add log-probability to the previous score
                    new_seq = seq + [word_idx.item()]

                    all_candidates.append((self.embedding(word_idx.unsqueeze(0)), h, c, new_seq, new_score))

            # Sort all candidates by score (descending) and update beams
            beams = sorted(all_candidates, key=lambda x: x[4], reverse=True)[:beam_size]

        # The beam with the highest score will contain the best caption
        best_beam = beams[0]
        caption = best_beam[3]  # Get the sequence of words with the highest score

        # Convert the sequence of word indices to words
        caption_words = [vocab.itos[idx] for idx in caption]

        return caption_words, alphas  # caption_words is the generated caption, alphas are the attention weights at each timestep



    def init_hidden_state(self, encoder_out):
        mean_encoder_out = encoder_out.mean(dim=1) # tính trung bình theo column ==> output kích cỡ là (batch_size,2048)
        h = self.init_h(mean_encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c(mean_encoder_out)
        return h, c


In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()
        self.encoder = EncoderCNN()
        self.decoder = DecoderRNN(
            embed_size=embed_size,
            vocab_size = len(dataset.vocab),
            attention_dim=attention_dim,
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim
        )

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs # (batch_size,seq_length,vocab_size).
        # output có số row = độ dài câu và mỗi câu là giá trị softmax các từ phù hợp


### 4) Setting Hypperparameter and Init the model

In [ ]:
#Hyperparams
embed_size=300
vocab_size = len(dataset.vocab)
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 3e-4


In [ ]:
#init model
model = EncoderDecoder(
    embed_size=300,
    vocab_size = len(dataset.vocab),
    attention_dim=256,
    encoder_dim=2048,
    decoder_dim=512
).to(device)


criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#helper function to save the model
def save_model(model, num_epochs):
    model_state = {
        'num_epochs': num_epochs,              # Số epoch đã huấn luyện
        'embed_size': embed_size,              # Kích thước embedding
        'vocab_size': len(dataset.vocab),      # Kích thước từ điển
        'attention_dim': attention_dim,        # Kích thước attention
        'encoder_dim': encoder_dim,            # Kích thước đầu ra encoder
        'decoder_dim': decoder_dim,            # Kích thước hidden state trong decoder
        'state_dict': model.state_dict()       # Trạng thái các tham số của mô hình
    }

    torch.save(model_state, 'attention_LSTM_model_state.pth')


## 5) Training Job from above configs

In [ ]:
num_epochs = 40
print_every = 200

for epoch in range(1,num_epochs+1):
    for idx, (image, captions) in enumerate(iter(data_loader)):
        image,captions = image.to(device),captions.to(device)

        # Zero the gradients.
        optimizer.zero_grad()

        # Feed forward
        outputs,attentions = model(image, captions)

        # Calculate the batch loss.
        targets = captions[:,1:]
        loss = criterion(outputs.view(-1, vocab_size), targets.reshape(-1))

        # Backward pass.
        loss.backward()

        # Update the parameters in the optimizer.
        optimizer.step()

        if (idx+1)%print_every == 0:
            print("Epoch: {} loss: {:.5f}".format(epoch,loss.item()))


            #generate the caption
            model.eval()
            with torch.no_grad():
                dataiter = iter(data_loader)
                img,_ = next(dataiter)
                features = model.encoder(img[0:1].to(device))
                caps,alphas = model.decoder.generate_caption(features,vocab=dataset.vocab)
                caption = ' '.join(caps)
                show_image(img[0],title=caption)

            model.train()

    #save the latest model
    save_model(model,epoch)

## 6 Visualizing the attentions
Defining helper functions
<li>Given the image generate captions and attention scores</li>
<li>Plot the attention scores in the image</li>

In [ ]:
#generate caption
def get_caps_from(features_tensors):
    #generate the caption
    model.eval()
    with torch.no_grad():
        features = model.encoder(features_tensors.to(device))
        caps,alphas = model.decoder.generate_caption(features,vocab=dataset.vocab)
        caption = ' '.join(caps)
        show_image(features_tensors[0],title=caption)

    return caps,alphas

#Show attention
def plot_attention(img, result, attention_plot):
    #untransform
    img[0] = img[0] * 0.229
    img[1] = img[1] * 0.224
    img[2] = img[2] * 0.225
    img[0] += 0.485
    img[1] += 0.456
    img[2] += 0.406

    img = img.numpy().transpose((1, 2, 0))
    temp_image = img

    fig = plt.figure(figsize=(15, 15))

    len_result = len(result)
    for l in range(len_result):
        temp_att = attention_plot[l].reshape(7,7)

        ax = fig.add_subplot(len_result//2,len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.7, extent=img.get_extent())


    plt.tight_layout()
    plt.show()

In [ ]:
#show any 1
dataiter = iter(data_loader)
images,_ = next(dataiter)

img = images[0].detach().clone()
img1 = images[0].detach().clone()
caps,alphas = get_caps_from(img.unsqueeze(0))

plot_attention(img1, caps, alphas)

In [ ]:
#show any 1
dataiter = iter(data_loader)
images,_ = next(dataiter)

img = images[0].detach().clone()
img1 = images[0].detach().clone()
caps,alphas = get_caps_from(img.unsqueeze(0))

plot_attention(img1, caps, alphas)

In [ ]:
#show any 1
dataiter = iter(data_loader)
images,_ = next(dataiter)

img = images[0].detach().clone()
img1 = images[0].detach().clone()
caps,alphas = get_caps_from(img.unsqueeze(0))

plot_attention(img1, caps, alphas)

In [ ]:
#show any 1
dataiter = iter(data_loader)
images,_ = next(dataiter)

img = images[0].detach().clone()
img1 = images[0].detach().clone()
caps,alphas = get_caps_from(img.unsqueeze(0))

plot_attention(img1, caps, alphas)